# Setup

In [1]:
# Project Root directory
project_root = ".."

In [2]:
import random, os
import numpy as np

if "pipeline" not in os.listdir():
    os.chdir(project_root)
    assert "pipeline" in os.listdir()

from pipeline.preprocessing import \
    build_feature_extractor, \
    TCDPdata, \
    gen_datesets, \
    cross_train

# constant
dataset_root = "assets/the-circor-digiscope-phonocardiogram-dataset-1.0.3"

# Variables

In [3]:
from cnn2d import CNN2D

class args:
    
    cutoff_frequency = 2000 # use 0 to disable bandpass filter

    use_features = [
        "mel_2d"
    ]

    use_X = [
        "raw",
        "scaled",
        "minmax",
    ]

    train_size = 0.8

    random_state = 2024

    use_models = {
        "CNN2D": {
            "class": CNN2D,
            "kwargs": {
                "input_shape": [128, None, 1],  # (n_mels, time_frames, channels)
                "num_classes": 2,
                "learning_rate": 0.001,
                "epochs": 10,
                "batch_size": 32,
            }
        }
    }

    def set_n_time_frames(n_time_frames: int):
        args.use_models["CNN2D"]["kwargs"]['input_shape'][1] = n_time_frames

random.seed(args.random_state)
np.random.seed(args.random_state)

# Extract Features

In [5]:
extract_features = build_feature_extractor(
    args.use_features,
    args.cutoff_frequency
)
features, labels = TCDPdata(dataset_root).getXy(extract_features)

## defensive checking
# assert len(labels) == 3159
# assert sum(labels) == 1632
print('n features:', features.shape[1])
print('mean of labels:', labels.mean())

100%|██████████| 3159/3159 [02:33<00:00, 20.64it/s]


n features: 128
mean of labels: 0.51661918328585


# Generate Dataset

In [7]:
# features matrix has 3 dimensions: 0 for files, 1 for channels, 2 for time
# We want to normalize each channel(axis=1) separately,
# so the normalizer will move along axis 0 and 2. (normalize_axis=(0, 2))
X, y = gen_datesets(features, labels, args.use_X, args.train_size, args.random_state,
                    normalize_axis=(0, 2))

for x_type in X.keys():
    print(x_type, X[x_type]['train'].shape, X[x_type]['test'].shape)

raw (2527, 128, 505) (632, 128, 505)
scaled (2527, 128, 505) (632, 128, 505)
minmax (2527, 128, 505) (632, 128, 505)


In [8]:
n_time_frames = X['raw']['train'].shape[2]
args.set_n_time_frames(n_time_frames)
print(f"Model has aligned its input layer with n_time_frames: {n_time_frames}")

Model has aligned its input layer with n_time_frames: 505


# Train Models

In [9]:
"""
In case you run the next cell accidently,
which can make you lose all the data.
You need to run the cell first before the next one.
"""
models = {}
scores = {}

In [10]:
assert len(scores) == len(models) == 0, "rerun the cell above to start a new experiment"
assert len(args.use_X) > 0 and len(args.use_models) > 0, "at least one pair of train/test sets and one model is required"

models, scores = cross_train(X, y, args.use_models)

Dataset: raw, Model: CNN2D, Training...


d:\python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.5155 - loss: 2.5553
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.5571 - loss: 0.6758
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 125s 2s/step - accuracy: 0.6008 - loss: 0.6576
Epoch 4/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.6335 - loss: 0.6233
Epoch 5/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.6715 - loss: 0.6233
Epoch 6/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.7469 - loss: 0.5294
Epoch 7/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.7677 - loss: 0.4662
Epoch 8/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 115s 1s/step - accuracy: 0.7731 - loss: 0.5517
Epoch 9/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.7699 - loss: 0.5370
Epoch 10/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 115s 1s/step - accuracy: 0.8083 - loss: 0.4161
79/79 ━━━━━━━━━━━━━━━━━━━━ 26s 328ms/step
Performance on train set:
{'accuracy': 0.7981796860694885,
 'auc': 0.8030785302864888,
 'f1': 0.77883781439

d:\python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 114s 1s/step - accuracy: 0.4936 - loss: 0.6972
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 120s 2s/step - accuracy: 0.5205 - loss: 0.6925
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.5654 - loss: 0.6814
Epoch 4/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 118s 1s/step - accuracy: 0.5720 - loss: 0.6679
Epoch 5/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - accuracy: 0.5788 - loss: 0.6471
Epoch 6/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.5855 - loss: 0.6301
Epoch 7/10
25/79 ━━━━━━━━━━━━━━━━━━━━ 1:12 1s/step - accuracy: 0.6238 - loss: 0.6076

In [ ]:
scores